In [1]:
#!/usr/bin/env python3
"""
ParaView Python script to create slices perpendicular to the y-axis
from three Tecplot .dat files and generate plots.
"""

import os
from paraview.simple import *

def create_y_axis_slices(tecplot_files, output_dir="output", slice_positions=None):
    """
    Create slices perpendicular to the y-axis from Tecplot files using ParaView.
    
    Args:
        tecplot_files (list): List of paths to .tecplot.dat files
        output_dir (str): Directory to save output images
        slice_positions (list): Y-coordinates for slice positions. If None, uses default positions.
    """
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Default slice positions if none provided
    if slice_positions is None:
        slice_positions = [0.0, 0.25, 0.5, 0.75, 1.0]
    
    # Process each Tecplot file
    for i, tecplot_file in enumerate(tecplot_files):
        print(f"Processing file {i+1}/{len(tecplot_files)}: {tecplot_file}")
        
        # Check if file exists
        if not os.path.exists(tecplot_file):
            print(f"Warning: File {tecplot_file} not found. Skipping...")
            continue
        
        # Clear any existing sources
        Delete(GetSources())
        
        # Read the Tecplot file
        try:
            tecplot_reader = TecplotReader(FileName=tecplot_file)
            tecplot_reader.UpdatePipeline()
            
            # Get data information
            data_info = tecplot_reader.GetDataInformation()
            bounds = data_info.GetBounds()
            y_min, y_max = bounds[2], bounds[3]  # Y bounds are at indices 2,3
            
            print(f"Y-axis bounds: [{y_min:.3f}, {y_max:.3f}]")
            
            # Adjust slice positions to data bounds if using defaults
            if slice_positions == [0.0, 0.25, 0.5, 0.75, 1.0]:
                actual_positions = [y_min + (y_max - y_min) * pos for pos in slice_positions]
            else:
                actual_positions = slice_positions
            
            # Create slices at different y-positions
            for j, y_pos in enumerate(actual_positions):
                print(f"  Creating slice at y = {y_pos:.3f}")
                
                # Create slice filter
                slice_filter = Slice(Input=tecplot_reader)
                slice_filter.SliceType = 'Plane'
                slice_filter.SliceType.Origin = [0.0, y_pos, 0.0]
                slice_filter.SliceType.Normal = [0.0, 1.0, 0.0]  # Normal to y-axis
                slice_filter.UpdatePipeline()
                
                # Create a new render view
                render_view = CreateView('RenderView')
                render_view.ViewSize = [1200, 800]
                render_view.Background = [1.0, 1.0, 1.0]  # White background
                
                # Create display representation
                slice_display = Show(slice_filter, render_view)
                
                # Set up coloring (assuming first variable for coloring)
                available_arrays = slice_filter.PointData.keys()
                if available_arrays:
                    color_array = available_arrays[0]
                    ColorBy(slice_display, ('POINTS', color_array))
                    
                    # Add color bar
                    color_bar = GetScalarBar(GetColorTransferFunction(color_array), render_view)
                    color_bar.Title = color_array
                    color_bar.Visibility = 1
                
                # Set up the view
                render_view.ResetCamera()
                render_view.Update()
                
                # Save screenshot
                base_name = os.path.splitext(os.path.basename(tecplot_file))[0]
                output_filename = f"{base_name}_y_slice_{j+1}_pos_{y_pos:.3f}.png"
                output_path = os.path.join(output_dir, output_filename)
                
                SaveScreenshot(output_path, render_view, ImageResolution=[1200, 800])
                print(f"  Saved: {output_path}")
                
                # Clean up for next iteration
                Delete(render_view)
                Delete(slice_filter)
                
        except Exception as e:
            print(f"Error processing {tecplot_file}: {str(e)}")
            continue
    
    print("Processing complete!")

def main():
    """
    Main function to set up file paths and run the slicing process.
    """
    
    # Define your three Tecplot files here
    tecplot_files = [
        "0001.tecplot.dat",
        "0002.tecplot.dat", 
        "0003.tecplot.dat"
    ]
    
    # Optional: Define custom slice positions (y-coordinates)
    # If None, will use 5 evenly spaced slices across the y-range
    custom_slice_positions = None  # [0.1, 0.3, 0.5, 0.7, 0.9]
    
    # Output directory for images
    output_directory = "paraview_slices"
    
    # Create the slices
    create_y_axis_slices(
        tecplot_files=tecplot_files,
        output_dir=output_directory,
        slice_positions=custom_slice_positions
    )

if __name__ == "__main__":
    # Initialize ParaView
    paraview.simple._DisableFirstRenderCameraReset()
    
    # Run the main function
    main()
    
    print("\nScript completed successfully!")
    print("Check the output directory for generated slice images.")


# Additional utility functions

def batch_process_directory(directory_path, pattern="*.tecplot.dat"):
    """
    Process all Tecplot files in a directory matching the given pattern.
    
    Args:
        directory_path (str): Path to directory containing Tecplot files
        pattern (str): File pattern to match (default: "*.tecplot.dat")
    """
    import glob
    
    search_pattern = os.path.join(directory_path, pattern)
    tecplot_files = glob.glob(search_pattern)
    
    if len(tecplot_files) == 0:
        print(f"No files found matching pattern: {search_pattern}")
        return
    
    print(f"Found {len(tecplot_files)} Tecplot files:")
    for f in tecplot_files:
        print(f"  - {f}")
    
    create_y_axis_slices(tecplot_files)

def create_animation_slices(tecplot_file, num_slices=20, output_dir="animation_frames"):
    """
    Create multiple slices for animation purposes.
    
    Args:
        tecplot_file (str): Path to single Tecplot file
        num_slices (int): Number of slices to create
        output_dir (str): Directory for output frames
    """
    
    # Read file to get bounds
    tecplot_reader = TecplotReader(FileName=tecplot_file)
    tecplot_reader.UpdatePipeline()
    
    data_info = tecplot_reader.GetDataInformation()
    bounds = data_info.GetBounds()
    y_min, y_max = bounds[2], bounds[3]
    
    # Create evenly spaced slice positions
    slice_positions = [y_min + (y_max - y_min) * i / (num_slices - 1) 
                      for i in range(num_slices)]
    
    create_y_axis_slices([tecplot_file], output_dir, slice_positions)
    
    Delete(tecplot_reader)

ModuleNotFoundError: No module named 'paraview'